In [1]:
from vplants.autowig import autowig

In [2]:
from path import path

rootdir = path('../../').abspath()

includedir = rootdir + "/core/build-scons/include"
headers = [str(f) for f in includedir.walkfiles('*.h')]
    
flags = ['-x', 'c++', '-g', '-std=c++11', '-stdlib=libstdc++',
         '-I/usr/include', '-I' + str(includedir), '-D__STDC_CONSTANT_MACROS',
         '-I/usr/local/lib/clang/3.7.0/include', '-D__STDC_LIMIT_MACROS']

force = True

asg = autowig.AbstractSemanticGraph()
autowig.front_end.plugin = 'pyclanglite'
autowig.front_end(asg, headers, flags=flags, silent=True,
                  force_overload=True, bootstrap=3, cache=rootdir+'/.AutoWIG', force=force)

In [7]:
cls = asg['class ::std::vector<boost::io::detail::format_item<char, std::char_traits<char>, std::allocator<char> >, std::allocator<boost::io::detail::format_item<char, std::char_traits<char>, std::allocator<char> > > >']
cls

class ::std::vector<boost::io::detail::format_item<char, std::char_traits<char>, std::allocator<char> >, std::allocator<boost::io::detail::format_item<char, std::char_traits<char>, std::allocator<char> > > >

In [3]:
autowig.middle_end.plugin = 'default'
autowig.middle_end(asg)

primaries = [header for header in asg.files(header=True) if header.is_primary]
for hdr in asg.files(header=True):
    if not hdr.is_primary and hdr.include in primaries:
        hdr.is_primary = True
        
for inc in asg.directories():
    if inc.as_include:
        if inc.globalname + 'c++/' in asg:
            for vsn in asg[inc.globalname + 'c++/'].directories:
                vsn.as_include = True
                for hdr in vsn.files:
                    if hdr.suffix == '':
                        hdr.is_primary = True
        elif inc.globalname + 'boost/' in asg:
            for hdr in sg[inc.globalname + 'boost/'].files:
                hdr.is_primary = True
                
for tpl in asg.classes('.*(unique|weak|shared)_ptr$', templated=True,
                       specialized=False):
    tpl.is_smart_pointer = True

In [4]:
autowig.back_end.plugin = 'boost_python:std_filter'
autowig.back_end(asg)

In [5]:
if 'struct ::arma::Base<double, arma::Op<arma::diagview<double>, arma::op_strans> >' in asg:
    asg['struct ::arma::Base<double, arma::Op<arma::diagview<double>, arma::op_strans> >'].boost_python_export = False
if 'struct ::arma::Base<double, arma::Op<arma::Row<double>, arma::op_strans> >' in asg:
    asg['struct ::arma::Base<double, arma::Op<arma::Row<double>, arma::op_strans> >'].boost_python_export = False
if 'struct ::arma::Base<double, arma::Op<arma::subview_col<double>, arma::op_strans> >':
    asg['struct ::arma::Base<double, arma::Op<arma::subview_col<double>, arma::op_strans> >'].boost_python_export = False
if 'struct ::arma::Base<double, arma::Op<arma::subview_row<double>, arma::op_strans> >':
    asg['struct ::arma::Base<double, arma::Op<arma::subview_row<double>, arma::op_strans> >'].boost_python_export = False
if 'struct ::arma::Base<double, arma::Op<arma::Col<double>, arma::op_strans> >' in asg:
    asg['struct ::arma::Base<double, arma::Op<arma::Col<double>, arma::op_strans> >'].boost_python_export = False
if 'class ::arma::SpMat<double>' in asg:
    for mtd in asg['class ::arma::SpMat<double>'].methods(r'.*operator\*=.*'):
        if mtd.parameters[0].type.target == 'class ::arma::subview<double>':
            mtd.boost_python_export = False
if 'class ::arma::SpSubview<double>' in asg:
    for mtd in asg['class ::arma::SpSubview<double>'].methods('.*check_overlap.*'):
        mtd.boost_python_export = False
if 'class ::arma::SpMat<unsigned long long>' in asg:
    for mtd in asg['class ::arma::SpMat<unsigned long long>'].methods(r'.*operator\*=.*'):
        if mtd.parameters[0].type.target == 'class ::arma::subview<unsigned long long>':
            mtd.boost_python_export = False
if 'class ::arma::SpSubview<unsigned long long>' in asg:
    for mtd in asg['class ::arma::SpSubview<unsigned long long>'].methods('.*check_overlap.*'):
        mtd.boost_python_export = False

In [6]:
autowig.back_end.plugin = 'boost_python:export'
autowig.back_end(asg,
                 directory = rootdir + '/core/src/wrapper',
                 pattern = '.*statiskit.*',
                 prefix = '_')
autowig.back_end.plugin = 'boost_python:closure'
autowig.back_end(asg)
autowig.back_end.plugin = 'boost_python:export'
autowig.back_end(asg,
                 directory = rootdir + '/misc/src/wrapper',
                 prefix = '_')
autowig.back_end.plugin = 'boost_python:module'
autowig.back_end(asg,
                 filename = rootdir + '/core/src/wrapper/__core.cpp',
                 package = 'statiskit.core')
autowig.back_end(asg,
                 filename = rootdir + '/misc/src/wrapper/__misc.cpp',
                 package = 'statiskit.misc')
autowig.back_end.plugin = 'boost_python:import'
autowig.back_end(asg,
                 filename = rootdir + '/core/src/statiskit/core/_core.py',
                 module = rootdir + '/core/src/wrapper/__core.cpp')
autowig.back_end(asg,
                 filename = rootdir + '/misc/src/statiskit/misc/_misc.py',
                 module = rootdir + '/misc/src/wrapper/__misc.cpp')

In [7]:
autowig.back_end.plugin = 'on_disk'
autowig.back_end(asg,
                 pattern = rootdir + '/(core|misc)/src/(wrapper|statiskit)/.*')

In [8]:
autowig.scons(rootdir + '/misc', '-j3')

In [9]:
autowig.scons(rootdir + '/core', '-j3')

In [10]:
asg['class ::std::vector<boost::io::detail::format_item<char, std::char_traits<char>, std::allocator<char> >, std::allocator<boost::io::detail::format_item<char, std::char_traits<char>, std::allocator<char> > >']

KeyError: path(u"'class ::std::vector<boost::io::detail::format_item<char, std::char_traits<char>, std::allocator<char> >, std::allocator<boost::io::detail::format_item<char, std::char_traits<char>, std::allocator<char> > >' parameter")

In [5]:
cls.header

/usr/include/c++/4.9/bits/stl_vector.h

In [15]:
cls.node

'class ::std::vector<boost::io::detail::format_item<char, std::char_traits<char>, std::allocator<char> >, std::allocator<boost::io::detail::format_item<char, std::char_traits<char>, std::allocator<char> > > >'

In [25]:
for node, edge in asg._type_edges.iteritems():
    if edge['target'] == cls.node:
        print node
for node, edges in asg._parameter_edges.iteritems():
    if any(edge['target'] == cls.node for edge in edges):
        print node
for node, edges in asg._template_edges.iteritems():
    if any(edge['target'] == cls.node for edge in edges):
        print node

::std::vector<boost::io::detail::format_item<char, std::char_traits<char>, std::allocator<char> >, std::allocator<boost::io::detail::format_item<char, std::char_traits<char>, std::allocator<char> > > >::operator=::b1131a5f-6c5a-5cb5-b83f-3e838a7ea3e4
::std::vector<boost::io::detail::format_item<char, std::char_traits<char>, std::allocator<char> >, std::allocator<boost::io::detail::format_item<char, std::char_traits<char>, std::allocator<char> > > >::operator=::109eeca9-2253-5382-ba86-e28f351234e2
::std::vector<boost::io::detail::format_item<char, std::char_traits<char>, std::allocator<char> >, std::allocator<boost::io::detail::format_item<char, std::char_traits<char>, std::allocator<char> > > >::operator=::be1801d7-d72a-565c-ab86-0131f57c91fe
::std::vector<boost::io::detail::format_item<char, std::char_traits<char>, std::allocator<char> >, std::allocator<boost::io::detail::format_item<char, std::char_traits<char>, std::allocator<char> > > >::vector::8938f37a-32e5-59b6-bdbd-4300faf72926

In [24]:
asg._template_edges

{'struct ::std::iterator_traits<const long double *>': [{'specifiers': ' const *',
   'target': '::long double'}],
 'struct ::arma::Base_inv<arma::subview<char>, false>': [{'specifiers': '',
   'target': 'class ::arma::subview<char>'},
  {'specifiers': '', 'target': '::bool'}],
 'struct ::boost::iterator_value<const wchar_t *>': [{'specifiers': ' const *',
   'target': '::wchar_t'}],
 'struct ::arma::arma_unsigned_integral_only<unsigned long long>': [{'specifiers': '',
   'target': '::unsigned long long'}],
 'class ::__gnu_cxx::__normal_iterator<std::shared_ptr<statiskit::UnivariateSampleSpace> *, std::vector<std::shared_ptr<statiskit::UnivariateSampleSpace>, std::allocator<std::shared_ptr<statiskit::UnivariateSampleSpace> > > >': [{'specifiers': ' *',
   'target': 'class ::std::shared_ptr<statiskit::UnivariateSampleSpace>'},
  {'specifiers': '',
   'target': 'class ::std::vector<std::shared_ptr<statiskit::UnivariateSampleSpace>, std::allocator<std::shared_ptr<statiskit::UnivariateSamp

In [31]:
stt = asg.nodes('.*boost::io::detail::format_item$').pop().specializations().pop()
for node, edge in asg._type_edges.iteritems():
    if edge['target'] == stt.node:
        print node
for node, edges in asg._parameter_edges.iteritems():
    if any(edge['target'] == stt.node for edge in edges):
        print node
for node, edges in asg._template_edges.iteritems():
    if any(edge['target'] == stt.node for edge in edges):
        print node

::std::allocator<boost::io::detail::format_item<char, std::char_traits<char>, std::allocator<char> > >::const_pointer
::__gnu_cxx::__normal_iterator<const boost::io::detail::format_item<char, std::char_traits<char>, std::allocator<char> > *, std::vector<boost::io::detail::format_item<char, std::char_traits<char>, std::allocator<char> >, std::allocator<boost::io::detail::format_item<char, std::char_traits<char>, std::allocator<char> > > > >::operator->::1d90aa77-7943-5ee1-b393-52dc71c49b77
::std::initializer_list<boost::io::detail::format_item<char, std::char_traits<char>, std::allocator<char> > >::end::a9e3098a-e4c5-59d7-b67b-2074f58dae51
::std::vector<boost::io::detail::format_item<char, std::char_traits<char>, std::allocator<char> >, std::allocator<boost::io::detail::format_item<char, std::char_traits<char>, std::allocator<char> > > >::pointer
::__gnu_cxx::__normal_iterator<boost::io::detail::format_item<char, std::char_traits<char>, std::allocator<char> > *, std::vector<boost::io::d